# Lesson_5_ex
### Exercise. Предобработка данных и разведывательный анализ

### METADATA:
    Имеются следующие данные о транзакциях в период с 01.12.2010 по 12.09.2011:
   
            InvoiceNo – номер транзакции
            StockCode – код товара
            Description – описание товара
            Quantity – количество единиц товара, добавленных в заказ
            InvoiceDate – дата транзакции 
            UnitPrice – цена за единицу товара
            CustomerID – id клиента
            Country – страна, где проживает клиент

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

### Ex_1:
1) Импортируйте данные data.csv.zip (в папке) с кодировкой ISO-8859-1. Запишите полученный датафрейм в retail, а названия колонок сохраните в переменную retail_columns.

In [3]:
path = 'C://Users/user/Karpov_courses/Lessons/Lesson_5/data.csv.zip'
retail = pd.read_csv(path, compression= 'zip', encoding= 'ISO-8859-1')   #compression= 'zip' - Default
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [31]:
retail_columns = list(retail.columns)
retail_columns

['InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country']

### Ex_2:
2) Проверьте, встречаются ли в данных повторяющиеся наблюдения, и в качестве ответа укажите их количество. Если они есть, то удалите их из retail.

In [9]:
sum(retail.duplicated())
print("Количество дубликатов: ", nrows)
# Количество дубликатов считает и sum(retail.duplicated())

Количество дубликатов:  5268


In [5]:
nrows_original = retail.shape[0]
nrows_original

541909

In [6]:
retail.drop_duplicates(inplace=True)
nrows_unique = retail.shape[0]
nrows_unique

536641

In [8]:
nrows = nrows_original - nrows_unique 
print(f"Количество удаленных из DF дубликатов: {nrows} шт.")

Количество удаленных из DF дубликатов: 5268 шт.


### Ex_3:
3) ⭐️Задание со звёздочкой!⭐️
           
Данные содержат в себе записи как и об успешных транзакциях, так и об отмененных. Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled)
 
Сколько всего заказов отменили пользователи?

In [10]:
retail\
    .InvoiceNo\
    .str.startswith('C').sum()    
# str.startswith('C').sum() - указывает начинается ли значение с указанного символа, а сумма подсчитала их кол-во

9251

### Ex_4:
4) Теперь отфильтруйте данные и оставьте в retail только те заказы, где Quantity > 0. В качестве ответа укажите число оставшихся строк


In [11]:
retail = retail[ retail.Quantity > 0]

print ('''Число строк в DF после фильтрации по Quantity > 0:
{} строк(и).'''.format(retail[ retail.Quantity > 0].shape[0]))

Число строк в DF после фильтрации по Quantity > 0:
526054 строк(и).


### Ex_5:5) 💀 Сложное задание! 💀
   
Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany).

     Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль.
     Запишите полученные id пользователей в germany_top (не весь датафрейм, только id).
     Идентификатор заказа – InvoiceNo.
     Для каждого заказа в данных может встречаться более 1 строки.



In [16]:
german_buyers = retail\
                    [retail.Country == 'Germany']\
                    .groupby('CustomerID', as_index = False)\
                    .agg({'InvoiceNo': pd.Series.nunique})\
                    .rename(columns={'InvoiceNo':'count_orders'})
german_buyers


,CustomerID,count_orders
0,12426.0,1
1,12427.0,3
2,12468.0,2
3,12471.0,30
4,12472.0,7
...,...,...
89,13814.0,2
90,13815.0,4
91,13816.0,2
92,13817.0,2


In [17]:
german_buyers.InvoiceNo.sort_values().iloc[german_buyers.shape[0] // 2]
# нахождение медианы: медианное значение количества заказов среди покупателей = 3 заказа (50 процентиль)

In [18]:
percentile_80_German_buyers = german_buyers.count_orders.quantile(q = 0.8)
# 80 процентиль количества покупок покупателей из германии 

In [19]:
germany_top = german_buyers[german_buyers.count_orders > percentile_80_German_buyers].CustomerID.reset_index(drop = True)
germany_top

0     12471.0
1     12474.0
2     12476.0
3     12481.0
4     12500.0
5     12524.0
6     12569.0
7     12600.0
8     12619.0
9     12621.0
10    12626.0
11    12647.0
12    12662.0
13    12705.0
14    12708.0
15    12709.0
16    12712.0
17    12720.0
Name: CustomerID, dtype: float64

### Ex_6:6) ⭐️Задание со звёздочкой!⭐️
   
Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в top_retail_germany



In [20]:
germany_top_list = list(germany_top)
germany_top_list

[12471.0,
 12474.0,
 12476.0,
 12481.0,
 12500.0,
 12524.0,
 12569.0,
 12600.0,
 12619.0,
 12621.0,
 12626.0,
 12647.0,
 12662.0,
 12705.0,
 12708.0,
 12709.0,
 12712.0,
 12720.0]

In [21]:
top_retail_germany = retail\
                        .query('CustomerID in @germany_top_list')
top_retail_germany

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany
...,...,...,...,...,...,...,...,...
541726,581570,22139,RETROSPOT TEA SET CERAMIC 11 PC,3,12/9/2011 11:59,4.95,12662.0,Germany
541727,581570,23077,DOUGHNUT LIP GLOSS,20,12/9/2011 11:59,1.25,12662.0,Germany
541728,581570,20750,RED RETROSPOT MINI CASES,2,12/9/2011 11:59,7.95,12662.0,Germany
541729,581570,22505,MEMO BOARD COTTAGE DESIGN,4,12/9/2011 11:59,4.95,12662.0,Germany


### Ex_7:
7) Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST? 
        
Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.
y



In [32]:
# Вариант 1
top_products_by_orders = top_retail_germany.StockCode.value_counts()
top_products_by_orders

StockCode
POST     213
22326     52
22328     38
22423     34
20719     30
        ... 
21564      1
22584      1
22561      1
21497      1
23562      1
Name: count, Length: 1158, dtype: int64

In [33]:
# Вариант 2
top_products_by_orders = top_retail_germany\
                            .groupby('StockCode', as_index = False)\
                            .agg({'InvoiceNo':'count'})\
                            .rename(columns = {'InvoiceNo':'count_orders'})\
                            .query(' StockCode != "POST" ')\
                            .sort_values('count_orders', ascending= False)
top_products_by_orders

,StockCode,count_orders
409,22326,52
411,22328,38
453,22423,34
45,20719,30
709,22961,28
...,...,...
350,22168,1
769,23104,1
768,23102,1
354,22174,1


In [23]:
top_count_products = top_retail_germany\
                                .groupby('StockCode')\
                                .agg({'InvoiceNo':'count'})\
                                .rename(columns = {'InvoiceNo':'count_orders'})\
                                .query(' StockCode != "POST" ')\
                                .sort_values('count_orders', ascending= False)\
                                .count_orders.reset_index(drop=True)\
                                .max()
top_count_products
# макс количество заказов одного продукта - выводим как переменную, чтобы в будущем выводить более одного продукта с таким кол-вом заказов

52

In [24]:
top_products_name = top_products_by_orders\
                                .query('count_orders == @top_count_products ')
top_products_name = list(top_products_name.StockCode)
top_products_name
# список из топ-продуктов, которые покупались больше всего

['22326']

### Ex_8:
    8) Вернемся к анализу полного датасета retail. 
    Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.


In [25]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [26]:
retail['Revenue'] = retail.Quantity * retail.UnitPrice
retail.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850.0,United Kingdom,15.30
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850.0,United Kingdom,25.50
7,536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850.0,United Kingdom,11.10
8,536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850.0,United Kingdom,11.10
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/2010 8:34,1.69,13047.0,United Kingdom,54.08


In [36]:
# Вариант 2
# создание колонки методом assign, т.е. можно создавать колонки сразу в блоке кода
retail = retail.assign(
                Revenue = retail.UnitPrice * retail.Quantity
)
retail


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,16.60


### Ex_9:
  9) Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа.

         В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue)

In [45]:
top_5_InvoiceNo_by_TotalRevenue = retail\
                                    .groupby('InvoiceNo', as_index = False)\
                                    .agg({'Revenue':'sum'})\
                                    .rename(columns = {'Revenue':'TotalRevenue'})\
                                    .sort_values('TotalRevenue', ascending= False)[:5]\
                                    .reset_index(drop = True)\
                                    .InvoiceNo
top_5_InvoiceNo_by_TotalRevenue 

0    581483
1    541431
2    574941
3    576365
4    556444
Name: InvoiceNo, dtype: object

In [28]:
n_1 = str(*top_5_InvoiceNo_by_TotalRevenue[0])
n_2 = str(*top_5_InvoiceNo_by_TotalRevenue[1])
n_3 = str(*top_5_InvoiceNo_by_TotalRevenue[2])
n_4 = str(*top_5_InvoiceNo_by_TotalRevenue[3])
n_5 = str(*top_5_InvoiceNo_by_TotalRevenue[4])

In [49]:
print('''Топ-5 InvoiceNo по общей сумме продаж:
{}.'''.format(str(n_1 + ', '+ n_2 + ', ' + n_3 + ', ' + n_4 + ', ' + n_5) )
     )

Топ-5 InvoiceNo по общей сумме продаж:
581483, 541431, 574941, 576365, 556444.


In [50]:
# Вариант 2 

sorted_revenues = retail\
        .groupby('InvoiceNo', as_index = False)\
        .agg({'Revenue':'sum'})\
        .rename(columns = {'Revenue':'TotalRevenue'})\
        .sort_values('TotalRevenue', ascending= False)[:5]
sorted_revenues

,InvoiceNo,TotalRevenue
20689,581483,168469.60
2202,541431,77183.60
17582,574941,52940.94
18251,576365,50653.91
9034,556444,38970.00


In [51]:
sorted_revenues.InvoiceNo.str.cat(sep = ', ')
# преобразование данных из DF в строку через запятую

'581483, 541431, 574941, 576365, 556444'

In [48]:
print('''Топ-5 InvoiceNo по общей сумме продаж:
{}.'''.format(sorted_revenues.InvoiceNo.str.cat(sep = ', ')))

Топ-5 InvoiceNo по общей сумме продаж:
581483, 541431, 574941, 576365, 556444.
